In [1]:
import os,re, tempfile

from dotenv import load_dotenv
from langchain import hub
from langchain.agents import load_tools
from langchain_experimental.tools import PythonREPLTool
from langchain.agents import create_react_agent, AgentExecutor
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain_openai import ChatOpenAI
from langchain.tools import ArxivQueryRun, WikipediaQueryRun
from langchain.utilities import ArxivAPIWrapper, WikipediaAPIWrapper

from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

from llamp.mp.agents import (
    MPSummaryExpert,
    MPThermoExpert,
    MPElasticityExpert,
    MPDielectricExpert,
    MPPiezoelectricExpert,
    MPMagnetismExpert,
    MPElectronicExpert,
    MPSynthesisExpert,
    MPStructureRetriever,
)
from llamp.atomate2.agents import MLFFMDAgent
# from llamp.arxiv.agents import ArxivAgent

load_dotenv()

MP_API_KEY = os.getenv("MP_API_KEY", None)

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY", None)
OPENAI_ORGANIZATION = os.getenv("OPENAI_ORGANIZATION", None)


# OPENAI_GPT_MODEL = "gpt-4-1106-preview"
OPENAI_GPT_MODEL = "gpt-4-0125-preview"
# OPENAI_GPT_MODEL = "gpt-3.5-turbo-1106"
# OPENAI_GPT_MODEL = "gpt-4-turbo"

In [2]:
top_llm = ChatOpenAI(
    temperature=0.0,
    model=OPENAI_GPT_MODEL,
    openai_api_key=OPENAI_API_KEY,
    openai_organization=OPENAI_ORGANIZATION,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
)

bottom_callback_handler = StreamingStdOutCallbackHandler()

bottom_llm = ChatOpenAI(
    temperature=0.0,
    model=OPENAI_GPT_MODEL,
    openai_api_key=OPENAI_API_KEY,
    openai_organization=OPENAI_ORGANIZATION,
    max_retries=5,
    streaming=True,
    callbacks=[bottom_callback_handler],
)


wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())
arxiv = ArxivQueryRun(api_wrapper=ArxivAPIWrapper())

tools = load_tools(["llm-math"], llm=bottom_llm)
tools += [PythonREPLTool(
    description=(
        "A Python shell. Use this to execute python commands. "
        "Input should be a valid python command. "
        "If you want to see the output of a value, you should print it out "
        "with `print(...)`."
        "If you write the output to a file, ALWAYS print the path to the file."
    )
)]
tools += [
    MPThermoExpert(llm=bottom_llm, mp_api_key=MP_API_KEY).as_tool(
        agent_kwargs=dict(return_intermediate_steps=False)
    ),
    MPElasticityExpert(llm=bottom_llm, mp_api_key=MP_API_KEY).as_tool(
        agent_kwargs=dict(return_intermediate_steps=False)
    ),
    MPDielectricExpert(llm=bottom_llm, mp_api_key=MP_API_KEY).as_tool(
        agent_kwargs=dict(return_intermediate_steps=False)
    ),
    MPMagnetismExpert(llm=bottom_llm, mp_api_key=MP_API_KEY).as_tool(
        agent_kwargs=dict(return_intermediate_steps=False)
    ),
    MPElectronicExpert(llm=bottom_llm, mp_api_key=MP_API_KEY).as_tool(
        agent_kwargs=dict(return_intermediate_steps=False)
    ),
    MPPiezoelectricExpert(llm=bottom_llm, mp_api_key=MP_API_KEY).as_tool(
        agent_kwargs=dict(return_intermediate_steps=False)
    ),
    MPSummaryExpert(llm=bottom_llm, mp_api_key=MP_API_KEY).as_tool(
        agent_kwargs=dict(return_intermediate_steps=False)
    ),
    MPSynthesisExpert(llm=bottom_llm, mp_api_key=MP_API_KEY).as_tool(
        agent_kwargs=dict(return_intermediate_steps=False)
    ),
    MPStructureRetriever(llm=bottom_llm, mp_api_key=MP_API_KEY).as_tool(
        agent_kwargs=dict(return_intermediate_steps=False)
    ),
    MLFFMDAgent(llm=bottom_llm).as_tool(
        agent_kwargs=dict(return_intermediate_steps=False)
    ),
    arxiv,
    wikipedia,
]

instructions = re.sub(
        r"\s+",
        " ",
        """You are a data-aware agent that can consult materials-related
    data through Materials Project (MP) database, atoamte2, arXiv, Wikipedia, and 
    a python REPL. If you get an error, debug your code and try again. Only use 
    the output of your code to answer the question. Ask user to clarify their 
    queries if needed. Please note that you don't have direct control over MP but 
    through multiple assistant agents to help you. You need to provide complete 
    context in the input for assistants to do their job. REFINE and CRITQUE the 
    output of the assistants if needed. You can end the conversation by saying "Bye".
    """,
    ).replace("\n", " ")
base_prompt = hub.pull("langchain-ai/react-agent-template")
prompt = base_prompt.partial(instructions=instructions)

conversational_memory = ConversationBufferWindowMemory(
    memory_key="chat_history", k=5, return_messages=True
)

agent = create_react_agent(top_llm, tools, prompt)
agent_executor = AgentExecutor(
    agent=agent, tools=tools, verbose=True,
    handle_parsing_errors=True,
    memory=conversational_memory,
)

/Users/la-cyrusyc/miniconda3/envs/llamp/lib/python3.11/site-packages/mp_api/client/mprester.py:230: UserWarning: mpcontribs-client not installed. Install the package to query MPContribs data, or construct pourbaix diagrams: 'pip install mpcontribs-client'
  warnings.warn(


In [3]:

pattern = re.compile(r'\b' + re.escape("Bye") + r'\b[.!?]*')

query = """
I would like you to help me run MD simulation with the follwing steps:

1. Retrieve the stable LiTaO3 structure from Materials Project
2. Create a 2x2x2 supercell and save the sturcture to a file
2. Use atomate2 with foundation ML forcefield MACE to run Nose-Hoover NVT simulation from 300K to 800K for 1 ps with time step of 5 fs
3. Save the trajectory file
4. Read the trajectory file and plot the temperature vs time
"""

# with tempfile.TemporaryDirectory() as tmpdir:
#     os.chdir(tmpdir)
while True:
    query = query or input("Query:")
    print(f"Query: {query}")
    response = agent_executor.invoke(
        {
            "input": query,
        }
    )
    if re.search(pattern, response["output"]):
        break
    query = None

Query: 
I would like you to help me run MD simulation with the follwing steps:

1. Retrieve the stable LiTaO3 structure from Materials Project
2. Create a 2x2x2 supercell and save the sturcture to a file
2. Use atomate2 with foundation ML forcefield MACE to run Nose-Hoover NVT simulation from 300K to 800K for 1 ps with time step of 5 fs
3. Save the trajectory file
4. Read the trajectory file and plot the temperature vs time



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: MPSummaryExpert
Action Input: LiTaO3Thought: Do I need to use a tool? Yes
Action: MPSummaryExpert
Action Input: LiTaO3

> Entering new AgentExecutor chain...
Action:
```
{
  "action": "search_materials_summary__get",
  "action_input": {
    "formula": "LiTaO3"
  }
}
```Action:
```
{
  "action": "search_materials_summary__get",
  "action_input": {
    "formula": "LiTaO3"
  }
}
```Error on search_materials_summary__get: `fields` must be specified in the query. Please provide more de

Retrieving SummaryDoc documents:   0%|          | 0/5 [00:00<?, ?it/s]

[{'nsites': 10, 'composition': {'Li': 2.0, 'Ta': 2.0, 'O': 6.0}, 'formula_pretty': 'LiTaO3', 'symmetry': {'crystal_system': 'Trigonal', 'symbol': 'R3c', 'number': 161, 'point_group': '3m', 'symprec': 0.1, 'version': '2.0.2'}, 'material_id': 'mp-3666'}, {'nsites': 20, 'composition': {'Li': 4.0, 'Ta': 4.0, 'O': 12.0}, 'formula_pretty': 'LiTaO3', 'symmetry': {'crystal_system': 'Orthorhombic', 'symbol': 'Pnma', 'number': 62, 'point_group': 'mmm', 'symprec': 0.1, 'version': '2.0.2'}, 'material_id': 'mp-1105280'}, {'nsites': 10, 'composition': {'Li': 2.0, 'Ta': 2.0, 'O': 6.0}, 'formula_pretty': 'LiTaO3', 'symmetry': {'crystal_system': 'Tetragonal', 'symbol': 'P4/mbm', 'number': 127, 'point_group': '4/mmm', 'symprec': 0.1, 'version': '2.0.2'}, 'material_id': 'mp-754345'}, {'nsites': 10, 'composition': {'Li': 2.0, 'Ta': 2.0, 'O': 6.0}, 'formula_pretty': 'LiTaO3', 'symmetry': {'crystal_system': 'Orthorhombic', 'symbol': 'Cmcm', 'number': 63, 'point_group': 'mmm', 'symprec': 0.1, 'version': '2.0

Retrieving SummaryDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

All retrieved structures are saved as extxyz files to the following paths: mp-3666.extxyz


> Finished chain.
{'input': 'mp-3666', 'output': 'All retrieved structures are saved as extxyz files to the following paths: mp-3666.extxyz'}Do I need to use a tool? Yes
Action: MLFFMDAgent
Action Input: Run MD simulation with MACE forcefield on LiTaO3 structure from file mp-3666.extxyz, create a 2x2x2 supercell, Nose-Hoover NVT from 300K to 800K for 1 ps with a timestep of 5 fs. Save the trajectory.Do I need to use a tool? Yes
Action: MLFFMDAgent
Action Input: Run MD simulation with MACE forcefield on LiTaO3 structure from file mp-3666.extxyz, create a 2x2x2 supercell, Nose-Hoover NVT from 300K to 800K for 1 ps with a timestep of 5 fs. Save the trajectory.

> Entering new AgentExecutor chain...
Action:
```
{
  "action": "MLFF MD",
  "action_input": {
    "run_mode": "local",
    "project": "llamp-atomate2",
    "atom_dict": "mp-3666.extxyz",
    "force_field_name": "MLFF.MACE",
    "time_step":

: 

In [ ]:
response